In [1]:
# === CONFIG ===
KAGGLE_USERNAME = "krisslarsen"
KAGGLE_KEY      = "3abae56fb66d7c6776ecc680a8ee5a34"  # paste your token here

DATASET_SLUG = "dansbecker/food-101"
DOWNLOAD_DIR = "/home/kristoffel/datasets/food101_test"  # change if you like


In [4]:
# Cell A1 — set creds (if not already saved)
KAGGLE_USERNAME = "krisslarsen"
KAGGLE_KEY      = "3abae56fb66d7c6776ecc680a8ee5a34"  # consider rotating the old one

import os, json
from pathlib import Path

os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME
os.environ["KAGGLE_KEY"] = KAGGLE_KEY

kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(parents=True, exist_ok=True)
(kaggle_dir / "kaggle.json").write_text(
    json.dumps({"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}), encoding="utf-8"
)
try:
    (kaggle_dir / "kaggle.json").chmod(0o600)
except Exception:
    pass


In [5]:
# Cell A2 — download Food-101 with progress + unzip
from kaggle.api.kaggle_api_extended import KaggleApi
from pathlib import Path

DOWNLOAD_DIR = Path("/home/kristoffel/datasets/food101_test")
DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

api = KaggleApi()
api.authenticate()

# force=True re-downloads if files exist, unzip=True extracts for you
api.dataset_download_files(
    "dansbecker/food-101",
    path=str(DOWNLOAD_DIR),
    force=False,
    quiet=False,
    unzip=True,
)

print("✅ Done. Saved to:", DOWNLOAD_DIR)


Dataset URL: https://www.kaggle.com/datasets/dansbecker/food-101


100%|██████████| 9.38G/9.38G [00:09<00:00, 1.07GB/s]


RuntimeError: An unexpected error occurred: . Please report this issue at https://www.github.com/kaggle/kaggle-api

In [3]:
import os, tarfile, zipfile, subprocess
from pathlib import Path

target = Path(DOWNLOAD_DIR).expanduser().resolve()
target.mkdir(parents=True, exist_ok=True)

def unzip_all_in_dir(d: Path):
    # unzip .zip
    for z in sorted(d.glob("*.zip")):
        print(f"📦 Unzipping {z.name}")
        with zipfile.ZipFile(z, "r") as zf:
            zf.extractall(d)
        z.unlink()
    # extract .tar, .tar.gz, .tgz, .tar.bz2, .tbz2, .tar.xz
    for t in sorted(list(d.glob("*.tar")) +
                    list(d.glob("*.tar.gz")) + list(d.glob("*.tgz")) +
                    list(d.glob("*.tar.bz2")) + list(d.glob("*.tbz2")) +
                    list(d.glob("*.tar.xz"))):
        print(f"📦 Extracting {t.name}")
        with tarfile.open(t, "r:*") as tf:
            tf.extractall(d)
        t.unlink()

# Build the kaggle command
cmd = ["kaggle", "datasets", "download", "-d", DATASET_SLUG, "-p", str(target), "-o"]
print("→", " ".join(cmd))

# Stream output so you see progress
env = os.environ.copy()
if os.path.expanduser("~/.local/bin") not in env.get("PATH", ""):
    env["PATH"] = env.get("PATH", "") + os.pathsep + os.path.expanduser("~/.local/bin")

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env)
for line in proc.stdout:
    print(line, end="")
ret = proc.wait()
if ret != 0:
    raise RuntimeError("Kaggle download failed. Check credentials and internet connectivity.")

# Unpack any archives that were downloaded
unzip_all_in_dir(target)
print("\n✅ Done.")
print("Saved to:", target)


→ kaggle datasets download -d dansbecker/food-101 -p /home/kristoffel/datasets/food101_test -o


KeyboardInterrupt: 

In [ ]:
from pathlib import Path
import os

def folder_size(p: Path) -> float:
    total = 0
    for f in p.rglob("*"):
        if f.is_file():
            total += f.stat().st_size
    return total / (1024**3)

p = Path(DOWNLOAD_DIR)
print("Folder:", p)
print("Approx size (GB):", round(folder_size(p), 2))
print("\nTop-level contents:")
for item in sorted(p.iterdir()):
    print(" -", item.name)
